# RTE+RRTMGP-NN training - SDumont

*Last edited: 2024-08-23*


**[WORK IN PROGRESS]**



This Notebook generates files containing the neural network (NN) model that is later used in the RTE+RRTMGP-NN model. The implementation uses TensorFlow and Python, and Fortran routines are used to generate the training data set. The idea is to replace the RRTMGP lookup tables with NN. The code and data from Ukkonen & Hogan (2023) need to be installed for the Notebook to work. And after training, the model is saved to disk using two file formats, HDF5 and netCDF. The main code (RTE+RRTMGP) is written in Fortan, including the part that makes the prediction from the NN model.

Based on:

- Ukkonen, P., & Hogan, R. J. (2023). Implementation of a machine-learned gas optics parameterization in the ECMWF Integrated Forecasting System: RRTMGP-NN 2.0. Geoscientific Model Development, 16(11), 3241–3261. https://doi.org/10.5194/gmd-16-3241-2023

- Most of the code and data was adapted from:
    - <https://github.com/peterukk/rte-rrtmgp-nn>
    - <https://doi.org/10.5281/zenodo.7413935>

- Additional information, code and data can be obtained from::
    - <https://doi.org/10.5281/zenodo.6576680>
    - <https://doi.org/10.5281/zenodo.7148329>
    - <https://zenodo.org/records/7413935>
    - <https://zenodo.org/records/7413952>
    - <https://zenodo.org/records/7852526>

Notes:

- Up to this point in development, the Fortran code to generate data does not compile on SDumont, but it is possible to use data generated on the local PC and transferred to SDdumont, and use TensorFlow to train the DNN.
- The dataset, due to its size, is not hosted in this repository, and must be downloaded from the links above, and placed in the corresponding directories.
- The documentation is described in the work of Ukkonen & Hogan.
- In this Notebook, the base directory where the code and data are located is `ukk23test01/`.
- The training, which uses TensorFlow, is located in the `examples/rrtmgp-nn-training/` directory.
- This Notebook itself is located outside the base directory, one level below.
- The machine has 16 GB RAM, an NVIDIA GPU with 4 GB VRAM, and a 2-core Intel processor.
- Training data is generated from RRTMGP.
- To perform the prediction, "g-point" vectors are used, containing:
    - LW
        - Planck fraction, absorption cross-section, or both
    - SW
        - Absorption cross-section, or Rayleigh cross-section
- Models are saved to `../../neural/data` with a file name containing the custom radiation scores.

Task List:

- [ ] Run the Fortran code, which generates data, on SDumont.

## Dataset

Due to space footprint, the datasets and their respective repositories are not hosted in this GitHub repository, and need to be recreated from the links above. Some data files are created during the execution of the routines, such as the NN training data set. The data directories are:

- neural/data
- rrtmgp/data
- examples/rfmip-clear-sky/data
- examples/rfmip-clear-sky/output_fluxes
- examples/rrtmgp-nn-training/data
- examples/rrtmgp-nn-training/inputs_to_RRTMGP

## Dependencies

Fortran and dependencies:

    gfortran libopenblas-dev libnetcdf-dev libnetcdff-dev

Jupyter config

    module load gcc/11.1 netcdf/4.9.2_openmpi-4.1.6_gnu openblas/0.3.19_gnu cmake/3.9.4 make/4.3
    source $HOME/conda/bin/activate base
    jupyter-lab ...

Python dependencies:
  - python
  - numba
  - tbb
  - netcdf4
  - pip:
    - tensorflow

## Go to work dir

In [1]:
%cd ukk23test01/examples/rrtmgp-nn-training/

/prj/ampemi/eduardo.miranda2/ecrad-proposal/ukk23test01/examples/rrtmgp-nn-training


## Build the Fortran code

Environment variables configuration:

In [2]:
! make clean

VAR="../../"
rm rrtmgp_lw_gendata_rfmipstyle rrtmgp_sw_gendata_rfmipstyle *.o *.mod *.optrpt
rm: cannot remove ‘rrtmgp_lw_gendata_rfmipstyle’: No such file or directory
rm: cannot remove ‘rrtmgp_sw_gendata_rfmipstyle’: No such file or directory
rm: cannot remove ‘*.optrpt’: No such file or directory
make: [Makefile:151: clean] Error 1 (ignored)


(...) locations of the netCDF C and Fortran libraries and module files as environmental variables (NCHOME and NFHOME)

In [3]:
%env FC=gfortran
%env FCFLAGS=-ffree-line-length-none -m64 -march=native -O3 -lcurl
%env NCHOME=/scratch/app/netcdf/4.9.2_openmpi-4.1.6_gnu
%env NFHOME=/scratch/app/netcdf/4.9.2_openmpi-4.1.6_gnu
%env BLASLIB=openblas

env: FC=gfortran
env: FCFLAGS=-ffree-line-length-none -m64 -march=native -O3 -lcurl
env: NCHOME=/scratch/app/netcdf/4.9.2_openmpi-4.1.6_gnu
env: NFHOME=/scratch/app/netcdf/4.9.2_openmpi-4.1.6_gnu
env: BLASLIB=openblas


It compiles on the local machine, but on the SDumont it is showing errors and does not compile:

In [ ]:
%%bash
module load gcc/11.1 netcdf/4.9.2_openmpi-4.1.6_gnu openblas/0.3.19_gnu
make

- Example of executable command format: `./rrtmgp_lw_gendata_rfmipstyle [block_size] [input file] [k-distribution file] [input-output file]`

Once built, the next step is to generate the training data.

### Block_size

- The block size is the number of columns to be computed at a time, and must be an integer such that the remainder of dividing `ncol*nexp` by `block_size` is zero.
- `block_size = 3` worked in all cases:

## Training data generation

Using Fortran executables

In [ ]:
%%bash
./rrtmgp_lw_gendata_rfmipstyle \
    3 \
    inputs_to_RRTMGP/inputs_Garand_BIG.nc \
    ../../rrtmgp/data/rrtmgp-data-lw-g128-210809.nc \
    data/ml_training_lw_g128_Garand_BIG.nc

In [35]:
%%bash
./rrtmgp_lw_gendata_rfmipstyle \
    3 \
    inputs_to_RRTMGP/inputs_AMON_ssp245_ssp585_2054_2100.nc \
    ../../rrtmgp/data/rrtmgp-data-lw-g128-210809.nc \
    data/ml_training_lw_g128_AMON_ssp245_ssp585_2054_2100.nc

 Usage: rrtmgp_rfmip_lw [block_size] [rfmip_file] [k-distribution_file] input_output file]
 input fileinputs_to_RRTMGP/inputs_AMON_ssp245_ssp585_2054_2100.nc                                                                             
 ncol:         420 nexp:         200 nlay:          19
 Doing        28000 blocks of size            3
 Calculation uses gases: water_vapor ozone carbon_dioxide methane nitrous_oxide oxygen nitrogen cfc11 cfc12 carbon_monoxide carbon_tetrachloride hcfc22 hfc143a hfc125 hfc23 hfc32 hfc134a cf4 
 min of play   100.000000     k_dist%get_press_min()   1.00518358    
 -------------------------------------------------------------------------
 starting clear-sky longwave computations
 Finished with computations!
 mean of flux_down is:   83.7984009    
 mean of flux_up is:   279.662506    
 -------------------------------------------------------------------------
 Attempting to save RRTMGP input/output to data/ml_training_lw_g128_AMON_ssp245_ssp585_2054_2100.nc  

In [36]:
%%bash
./rrtmgp_lw_gendata_rfmipstyle \
    3 \
    inputs_to_RRTMGP/inputs_CAMS_new_CKDMIPstyle.nc \
    ../../rrtmgp/data/rrtmgp-data-lw-g128-210809.nc \
    data/ml_training_lw_g128_CAMS_new_CKDMIPstyle.nc

 Usage: rrtmgp_rfmip_lw [block_size] [rfmip_file] [k-distribution_file] input_output file]
 input fileinputs_to_RRTMGP/inputs_CAMS_new_CKDMIPstyle.nc                                                                                     
 ncol:        1000 nexp:          42 nlay:          60
 Doing        14000 blocks of size            3
 Calculation uses gases: water_vapor ozone carbon_dioxide methane nitrous_oxide oxygen nitrogen cfc11 cfc12 carbon_monoxide carbon_tetrachloride hcfc22 hfc143a hfc125 hfc23 hfc32 hfc134a cf4 
 min of play   10.0000000     k_dist%get_press_min()   1.00518358    
 -------------------------------------------------------------------------
 starting clear-sky longwave computations
 Finished with computations!
 mean of flux_down is:   94.9205475    
 mean of flux_up is:   257.382263    
 -------------------------------------------------------------------------
 Attempting to save RRTMGP input/output to data/ml_training_lw_g128_CAMS_new_CKDMIPstyle.nc          

In [37]:
%%bash
./rrtmgp_lw_gendata_rfmipstyle \
    3 \
    inputs_to_RRTMGP/inputs_CKDMIP-MM-Big.nc \
    ../../rrtmgp/data/rrtmgp-data-lw-g128-210809.nc \
    data/ml_training_lw_g128_CKDMIP-MMM-Big.nc

 Usage: rrtmgp_rfmip_lw [block_size] [rfmip_file] [k-distribution_file] input_output file]
 input fileinputs_to_RRTMGP/inputs_CKDMIP-MM-Big.nc                                                                                            
 ncol:         243 nexp:          58 nlay:          52
 Doing         4698 blocks of size            3
 Calculation uses gases: water_vapor ozone carbon_dioxide methane nitrous_oxide oxygen nitrogen cfc11 cfc12 carbon_monoxide carbon_tetrachloride hcfc22 hfc143a hfc125 hfc23 hfc32 hfc134a cf4 
 min of play  0.504999995     k_dist%get_press_min()   1.00518358    
 -------------------------------------------------------------------------
 starting clear-sky longwave computations
 Finished with computations!
 mean of flux_down is:   39.1128159    
 mean of flux_up is:   281.537994    
 -------------------------------------------------------------------------
 Attempting to save RRTMGP input/output to data/ml_training_lw_g128_CKDMIP-MMM-Big.nc                

## Training using TensorFlow

Loading the libraries:

In [2]:
import tensorflow as tf
from tensorflow.keras import losses, optimizers

2024-08-23 11:20:28.037990: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Checks GPU availability:

In [3]:
print(tf.config.list_physical_devices("GPU"))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU')]


In [4]:
import os
import sys
import numpy as np

Load routines contained in code files in the working directory:

In [5]:
from ml_load_save_preproc import (
    save_model_netcdf,
    load_rrtmgp,
    scale_outputs_wrapper,
    preproc_pow_standardization_reverse,
    preproc_tau_to_crossection,
    preproc_minmax_inputs_rrtmgp,
)

In [6]:
from ml_scaling_coefficients import xcoeffs_all, input_names_all

In [7]:
from ml_trainfuncs_keras import (
    create_model_mlp,
    expdiff,
    hybrid_loss_wrapper,
)

## Configure predictand, NN complexity, etc

In [8]:
predictand = "lw_both"

In [9]:
scaling_method = "Ukkonen2020"  # only option currently

For `use_existing_input_scaling_coefficients` True is generally a safe choice, min max coefficients have been computed using a large dataset spanning both LGM (Last Glacial Maximum) and high future emissions scenarios. However, check that your scaled inputs fall somewhere in the 0-1 range. Negative values in particular might cause problems:

In [10]:
use_existing_input_scaling_coefficients = True

## Loss function, metrics

In [11]:
patience = 70
epochs = 200

Added "()" to MeanSquaredError as described in <https://stackoverflow.com/questions/62494412/typeerror-expected-float32-passed-to-parameter-y-of-op-equal-got-auto-of> :

In [12]:
lossfunc = losses.MeanSquaredError()
mymetrics = ["mean_absolute_error"]
expfirst = False

## Batch size and learning rate

In [13]:
lr = 0.001
batch_size = 2048

## NN Hyperparameters

Number of neurons in each hidden layer:

In [14]:
neurons = [72, 72]

Activation functions used after each layer: first the input layer, and then the hidden layers:

In [15]:
activ = ["softsign", "softsign", "linear"]

In [16]:
if np.size(activ) != np.size(neurons) + 1:
    print("Number of activations must be number of hidden layers + 1!")

Weight initializer: the default is probably an OK choice  (glorot):

In [17]:
initializer = "glorot_uniform"

## Routine for concatenating existing datasets containing raw inputs and outputs

In [18]:
def add_dataset(fpath, predictand, expfirst, x, y, col_dry, input_names, kdist,
                data_str):
    x_new, y_new, col_dry_new, input_names_new, kdist_new = load_rrtmgp(
        fpath, predictand, expfirst=expfirst)
    if not (kdist == kdist_new):
        print("Kdist does not match previous dataset!")
        return None
    if not (input_names == input_names_new):
        print("Input_names does not match previous dataset!")
        return None
    ns = x.shape[0]
    x = np.concatenate((x, x_new), axis=0)
    y = np.concatenate((y, y_new), axis=0)
    col_dry = np.concatenate((col_dry, col_dry_new), axis=0)
    print("{:.2e} samples previously, {:.2e} after adding data from: {}".format(
        ns, x.shape[0],
        fpath.split("/")[-1]))
    data_str = data_str + " , " + fpath.split("/")[-1]
    return x, y, col_dry, data_str

## Provide data containing inputs and outputs

- Profiles used:
    - Expanded Garand
    - GCM data (AMON_...)
    - CAMS data
    - Extended CKDMIP-Average-Maximum-Minimum profiles
- RFMIP ised used for validation.

The full dataset consumes a lot of RAM and VRAM

In [19]:
datadir = "data/"
fpath = datadir + "ml_training_lw_g128_Garand_BIG.nc"  # 0.6 GB
fpath2 = datadir + "ml_training_lw_g128_AMON_ssp245_ssp585_2054_2100.nc"  # 1.7 GB
fpath3 = datadir + "ml_training_lw_g128_CAMS_new_CKDMIPstyle.nc"  # 2.6 GB
fpath4 = datadir + "ml_training_lw_g128_CKDMIP-MMM-Big.nc"  # 0.8 GB

Load only the small ones, to fit RAM

Choose a small dataset just for testing (training using the full dataset requires a lot of RAM):

In [20]:
#fpaths = [fpath, fpath2, fpath3, fpath4]
fpaths = [fpath, fpath2, fpath4]

## Load data 

Load training data

In [21]:
%pwd

'/prj/ampemi/eduardo.miranda2/ecrad-proposal/ukk23test01/examples/rrtmgp-nn-training'

In [22]:
! ls /prj/ampemi/eduardo.miranda2/ecrad-proposal/ukk23test01/examples/rrtmgp-nn-training/data/ml_training_lw_g128_Garand_BIG.nc

/prj/ampemi/eduardo.miranda2/ecrad-proposal/ukk23test01/examples/rrtmgp-nn-training/data/ml_training_lw_g128_Garand_BIG.nc


In [23]:
x_tr_raw, y_tr_raw, col_dry_tr, input_names, kdist = load_rrtmgp(
    fpaths[0], predictand, expfirst=expfirst)

input_names found in file
there are 13524 profiles in this dataset (322 experiments, 42 columns)


In [24]:
data_str = fpath.split("/")[-1]
# data_str

The full training dataset is split into multiple files:

In [25]:
%%time
# We can have different datasets that we merge
for fpath in fpaths[1:]:
    x_tr_raw, y_tr_raw, col_dry_tr, data_str = add_dataset(
        fpath,
        predictand,
        expfirst,
        x_tr_raw,
        y_tr_raw,
        col_dry_tr,
        input_names,
        kdist,
        data_str,
    )

input_names found in file
there are 84000 profiles in this dataset (200 experiments, 420 columns)
5.68e+05 samples previously, 2.16e+06 after adding data from: ml_training_lw_g128_AMON_ssp245_ssp585_2054_2100.nc
input_names found in file
there are 14094 profiles in this dataset (58 experiments, 243 columns)
2.16e+06 samples previously, 2.90e+06 after adding data from: ml_training_lw_g128_CKDMIP-MMM-Big.nc
CPU times: user 1.78 s, sys: 3.75 s, total: 5.53 s
Wall time: 9.76 s


In [26]:
nx = x_tr_raw.shape[1]  # temperature + pressure + gases
ny = y_tr_raw.shape[1]  # number of g-points

In [27]:
shuffle = True

It can be a cell that consumes a lot of time and memory, depending on the dataset:

## Input and output scaling

In [28]:
%%time
if scaling_method != "Ukkonen2020":
    print("Only one type of pre-processing currently supported!")
else:
    # Input scaling - min-max
    if use_existing_input_scaling_coefficients:
        if xcoeffs_all == None:
            sys.exit("Input scaling coefficients (xcoeffs) missing!")
        (xmin_all, xmax_all) = xcoeffs_all
        # input_names loaded from file, describes inputs in order of x_tr_raw
        # input_names_all corresponds to xmin_all and xmax_all
        # Order of inputs may be different than in the existing coefficients,
        # account for that by indexing
        a = np.array(input_names_all)
        b = np.array(input_names)
        indices = np.where(b[:, None] == a[None, :])[1]
        xmin = xmin_all[indices]
        xmax = xmax_all[indices]
        x_tr = preproc_minmax_inputs_rrtmgp(x_tr_raw, (xmin, xmax))
    else:
        x_tr, xmin, xmax = preproc_minmax_inputs_rrtmgp(x_tr_raw)
        # Output scaling
        # first, do y = y / N if y is optical depth, to get cross-sections
        # then, square root scaling y: y=y**(1/nfac); cheaper and weaker version of
        # log scaling. nfac = 8 for cross-sections, 2 for Planck fraction
        # After this, use standard-scaling (not for Planck fraction)

    y_tr, ymean, ystd = scale_outputs_wrapper(y_tr_raw, col_dry_tr, predictand)

CPU times: user 1min 51s, sys: 8.19 s, total: 1min 59s
Wall time: 24.3 s


## I/O

RRTMGP-NN models are saved as NetCDF files which contain metadata describing how to obtain the physical outputs, as well as the training data

In [29]:
x_scaling_str = (
    "To get the required NN inputs, do the following: "
    "x(i) = log(x(i)) for i=pressure; "
    "x(i) = x(i)**(1/4) for i=H2O and O3; "
    "x(i) = (x(i) - xmin(i)) / (xmax(i) - xmin(i)) for all inputs"
)
y_scaling_str = (
    "Model predicts scaled cross-sections. Given the raw NN output y,"
    " do the following to obtain optical depth: "
    "y(igpt,j) = ystd(igpt)*y(igpt,j) + ymean(igpt); y(igpt,j) "
    "= y(igpt,j)**8; y(igpt,j) = y(igpt,j) * layer_dry_air_molecules(j)"
)

In [30]:
model_str = ""

Try to reduce memory consumption:

In [31]:
import gc

In [32]:
gc.collect()

35909

## TensorFlow Training

### Create and compile model

In [33]:
%%time
devstr = "/gpu:0"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

optim = optimizers.Adam(learning_rate=lr)

model = create_model_mlp(nx=nx,
                         ny=ny,
                         neurons=neurons,
                         activ=activ,
                         kernel_init=initializer)

model.compile(loss=lossfunc, optimizer=optim, metrics=mymetrics)

CPU times: user 1.85 s, sys: 1.14 s, total: 2.99 s
Wall time: 2.99 s


2024-08-23 11:22:09.496115: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-23 11:22:12.099472: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30971 MB memory:  -> device: 0, name: Tesla V100-PCIE-32GB, pci bus id: 0000:3b:00.0, compute capability: 7.0
2024-08-23 11:22:12.100943: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 30971 MB memory:  -> device: 1, name: Tesla V100-PCIE-32GB, pci bus id: 0000:5e:00.0, compute capability: 7.0
2024-08-23 11:22:12.102610: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:

### Start training

In [34]:
%%time
with tf.device(devstr):
    history = model.fit(
        x_tr,
        y_tr,
        epochs=epochs,
        batch_size=batch_size,
        shuffle=shuffle,
        verbose=1,
        callbacks=[],
    )
    history = history.history

Epoch 1/200


2024-08-23 11:22:43.127504: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7fbff0bd1be0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-08-23 11:22:43.127546: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): Tesla V100-PCIE-32GB, Compute Capability 7.0
2024-08-23 11:22:43.127554: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (1): Tesla V100-PCIE-32GB, Compute Capability 7.0
2024-08-23 11:22:43.127560: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (2): Tesla V100-PCIE-32GB, Compute Capability 7.0
2024-08-23 11:22:43.127566: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (3): Tesla V100-PCIE-32GB, Compute Capability 7.0
2024-08-23 11:22:43.133872: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
Numba: Attempt

   1/1415 [..............................] - ETA: 1:46:52 - loss: 0.5369 - mean_absolute_error: 0.5530

Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.


1415/1415 [==============================] - 8s 2ms/step - loss: 0.0414 - mean_absolute_error: 0.0885
Epoch 2/200
1415/1415 [==============================] - 4s 2ms/step - loss: 0.0061 - mean_absolute_error: 0.0335
Epoch 3/200
1415/1415 [==============================] - 4s 3ms/step - loss: 0.0036 - mean_absolute_error: 0.0242
Epoch 4/200
1415/1415 [==============================] - 4s 2ms/step - loss: 0.0026 - mean_absolute_error: 0.0200
Epoch 5/200
1415/1415 [==============================] - 3s 2ms/step - loss: 0.0020 - mean_absolute_error: 0.0173
Epoch 6/200
1415/1415 [==============================] - 4s 2ms/step - loss: 0.0017 - mean_absolute_error: 0.0156
Epoch 7/200
1415/1415 [==============================] - 3s 2ms/step - loss: 0.0014 - mean_absolute_error: 0.0145
Epoch 8/200
1415/1415 [==============================] - 4s 2ms/step - loss: 0.0012 - mean_absolute_error: 0.0137
Epoch 9/200
1415/1415 [==============================] - 3s 2ms/step - loss: 0.0011 - mean_absolute_

## Total parameters:

In [35]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 72)                1368      
                                                                 
 dense_1 (Dense)             (None, 72)                5256      
                                                                 
 dense_2 (Dense)             (None, 256)               18688     
                                                                 
Total params: 25,312
Trainable params: 25,312
Non-trainable params: 0
_________________________________________________________________


## Save model

- Save model to NN model directory `../../neural/data` after training.
- File name includes loss values, so shouldn't override anything.

### Get a descriptive filename for the model

In [38]:
comm = "test01"

In [35]:
neurons_str = (np.array2string(np.array(neurons)).strip("[]").replace(" ", "_"))
neurons_str

'72_72'

In [37]:
kdist

'rrtmgp-data-lw-g128-210809.nc'

In [36]:
source = kdist[12:].strip(".nc")
source

'lw-g128-210809'

In [39]:
fpath_keras = ("../../neural/data/" + source + "_" + predictand[3:] + "_" +
               neurons_str + "_" + comm + ".h5")
fpath_keras

'../../neural/data/lw-g128-210809_both_72_72_test01.h5'

In [41]:
fpath_netcdf = fpath_keras[:-3] + ".nc"
fpath_netcdf

'../../neural/data/lw-g128-210809_both_72_72_test01.nc'

### Saving model in both netCDF and HDF5 format

In [ ]:
# model.save(fpath_keras, save_format="h5")

In [42]:
save_model_netcdf(
    fpath_netcdf,
    model,
    activ,
    input_names,
    kdist,
    xmin,
    xmax,
    ymean,
    ystd,
    y_scaling_comment=y_scaling_str,
    x_scaling_comment=x_scaling_str,
    data_comment=data_str,
    model_comment=model_str,
)

## References

Ukkonen, P., & Hogan, R. J. (2023). Implementation of a machine-learned gas optics parameterization in the ECMWF Integrated Forecasting System: RRTMGP-NN 2.0. Geoscientific Model Development, 16(11), 3241–3261. https://doi.org/10.5194/gmd-16-3241-2023

Ukkonen, P., & Hogan, R. J. (2024). Twelve Times Faster yet Accurate: A New State-Of-The-Art in Radiation Schemes via Performance and Spectral Optimization. Journal of Advances in Modeling Earth Systems, 16(1), e2023MS003932. https://doi.org/10.1029/2023MS003932

Ukkonen, P., & Hogan, R. J. (2023). Fast computation of cloud 3D radiative effects in dynamical models by optimizing the ecRad scheme [Preprint]. Preprints. https://doi.org/10.22541/essoar.168298700.07329865/v1

Ukkonen, P. (2022). Improving the trade-off between accuracy and efficiency of atmospheric radiative transfer computations by using machine learning and code optimization. http://dx.doi.org/10.13140/RG.2.2.27880.03846

Ukkonen, P. (2022). Exploring Pathways to More Accurate Machine Learning Emulation of Atmospheric Radiative Transfer. Journal of Advances in Modeling Earth Systems, 14(4), e2021MS002875. https://doi.org/10.1029/2021MS002875

Yao, Y., Zhong, X., Zheng, Y., & Wang, Z. (2023). A Physics-Incorporated Deep Learning Framework for Parameterization of Atmospheric Radiative Transfer. Journal of Advances in Modeling Earth Systems, 15(5), e2022MS003445. https://doi.org/10.1029/2022MS003445


## Environment

In [ ]:
%%bash
source ${HOME}/conda/bin/activate tf2
conda export --file requirements.yml

Data:

In [ ]:
%%bash
ls -1 ukk23test01/neural/data/ > neur_data.txt
ls -1 ukk23test01/rrtmgp/data/ > rrtm_data.txt
ls -1 ukk23test01/examples/rfmip-clear-sky/data/ > exam_rfmi_data.txt
ls -1 ukk23test01/examples/rfmip-clear-sky/output_fluxes/ > exam_rfmi_flux.txt
ls -1 ukk23test01/examples/rrtmgp-nn-training/data/ > exam_rrtm_data.txt
ls -1 ukk23test01/examples/rrtmgp-nn-training/inputs_to_RRTMGP/ >  exam_rrtm_rrtm.txt